In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 20
LR = 0.006
def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (50batch, 20steps)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
    # plt.show()
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]

model = Sequential()
# build a LSTM RNN
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    units=CELL_SIZE,
    return_sequences=True,      # True: output at all steps. False: output as last step.
    stateful=True,              # True: the final state of batch1 is feed into the initial state of batch2
))
# add output layer
model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='mse',
           )

In [2]:
import plotly.graph_objects as go
import numpy as np
def saveres():
    X_test, y_test, xs = get_batch()
    res = model.predict(X_test)
    np.sum(abs(y_test.flatten()-res.flatten())**2)/len(y_test.flatten())
    m = go.Frame(data=[
                       go.Scatter(x=np.arange(len(y_test.flatten())), y=y_test.flatten()),
                       go.Scatter(x=np.arange(len(y_test.flatten())), y=res.flatten()),
                      ]
                )
    return m

In [3]:
def epochchange(epochs):
    print('Training ------------')
    X_train, y_train, xs = get_batch()
    h = []
    for i in range(epochs):
        history = model.fit(X_train, y_train,epochs=5, batch_size=50, verbose=0)
        h.append(history.history['loss'])
    print(np.array(h).mean())
    return np.array(h).mean(),saveres()

In [11]:
loss

[0.5040347218513489,
 0.4788836050033569,
 0.2536196501851082,
 0.14224316823482513]

In [4]:
epochs =[1,10,100,200]
res = []
loss = []
for i in epochs:
    loss_,res_ = epochchange(i)
    res.append(res_)
    loss.append(loss_)
N = res
# print(N)

Training ------------
0.5040347218513489
Training ------------
0.4788836050033569
Training ------------
0.2536196501851082
Training ------------
0.14224316823482513


In [10]:
import plotly.graph_objects as go

# 初始化数据
real = go.Scatter(
                    x =(0,),   # 定义坐标轴的映射关系，将world_rank这一列作为x轴
                    y = (0,),    # 同理，将citations这一列作为y轴
                    mode = "lines+markers",      # 我们要绘制折线图，所以将mode设置为“lines”
                    name = "real",  # 将这条折线命名为citations
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'), 
                    # maker用来定义点的性质，如颜色、大小等
                    text= "df.university_name")
                    # 将university_name一列设置为悬停文本（鼠标悬停在图片上方显示的内容）

# 设置第二条折线trace2
pred = go.Scatter(
                    x = (0,),
                    y = (0,),
                    mode = "lines+markers", #绘制的折线图由散点连接而成，即lines+markers
                    name = "pred",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "df.university_name")
data = [real, pred]

fig = go.Figure(
    data=data,
    layout=go.Layout(
        xaxis=dict(range=[0, 1000], autorange=False),
        yaxis=dict(range=[-1, 1], autorange=False),
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[
                 {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
            ]
    )]),
    frames=N)

fig.show()

In [12]:
import plotly.graph_objects as go
import numpy as np

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(len(loss)), y=loss,
mode='lines+markers',
name="loss"))
fig.show()

In [6]:
import plotly.express as px

df = px.data.gapminder()

fig = px.bar(df, x="continent", y="pop", color="continent",
  animation_frame="year", animation_group="country", range_y=[0,4000000000])
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder()
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])